<a href="https://colab.research.google.com/github/skywalker0803r/-AI-/blob/main/api%E4%BD%BF%E7%94%A8%E7%AF%84%E4%BE%8B_%E5%BD%B1%E7%89%87%E7%89%88api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
import requests
# 設定 API 的 URL
#可以ping 但不要太長ping 我怕免費額度爆掉
url = "https://mmpose-api-924124779607.us-central1.run.app/pose_video"  # 替換為你的 API URL
# 使用 with 開啟檔案並傳送
with open('/content/pitch_0029.mp4', 'rb') as video_file:
    # 以 multipart/form-data 格式上傳檔案
    files = {'file': ('/content/pitch_0029.mp4', video_file, 'video/mp4')}
    response = requests.post(url, files=files)
# 查看回應內容
print(response.json())
response.json()['frames']

# TODO 回傳的response.json() 丟到一個函數 計算出 所有需要的東西


In [ ]:
def response2features(response.json()):
  #算算算一大堆 例如運動力學特徵
  return

# 測試用 執行這個CELL 會把 response.json() 渲染到原始影片上

In [ ]:
import cv2
import base64
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# 為了在 Colab 中顯示保存的影片
from IPython.display import Video, display
import os # 用於檢查檔案是否存在

# COCO 關鍵點定義 (17 個點的順序)
# 確保這個列表與您的 API 返回的關鍵點順序一致
COCO_KEYPOINTS = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear",
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip", "left_knee",
    "right_knee", "left_ankle", "right_ankle"
]

# COCO 骨架連接關係 (基於 COCO_KEYPOINTS 的索引)
COCO_CONNECTIONS = [
    (0, 1), (0, 2), (1, 3), (2, 4),  # 頭部和臉: 鼻子-左眼, 鼻子-右眼, 左眼-左耳, 右耳-右耳
    (5, 6),  # 軀幹: 左肩-右肩
    (5, 7), (7, 9),  # 左臂: 左肩-左肘, 左肘-左手腕
    (6, 8), (8, 10), # 右臂: 右肩-右肘, 右肘-右手腕
    (5, 11), (6, 12), (11, 12), # 軀幹和臀部: 左肩-左髖, 右肩-右髖, 左髖-右髖
    (11, 13), (13, 15), # 左腿: 左髖-左膝, 左膝-左腳踝
    (12, 14), (14, 16)  # 右腿: 右髖-右膝, 右膝-右腳踝
]


def draw_pose_on_frame(frame, predictions, min_score_thresh=0.5, point_radius=5, line_thickness=2):
    """
    在單個影片幀上繪製姿態關鍵點和骨架。

    Args:
        frame (np.array): OpenCV 圖片幀 (BGR 格式)。
        predictions (list): API 回應中針對該幀的 'predictions' 列表。
                            預期格式: [{'keypoints': [[x, y], ...], 'keypoint_scores': [...], ...}]
        min_score_thresh (float): 關鍵點顯示的最低置信度閾值。
        point_radius (int): 關鍵點圓圈的半徑。
        line_thickness (int): 骨架線條的粗細。

    Returns:
        np.array: 繪製了姿態的幀 (BGR 格式)。
    """
    if not predictions:
        return frame # 如果沒有偵測到姿態，直接返回原幀

    # 針對每個偵測到的人繪製姿態
    for person_prediction in predictions:
        keypoints_coords = person_prediction.get('keypoints', [])
        keypoint_scores = person_prediction.get('keypoint_scores', [])

        if not keypoints_coords or not keypoint_scores:
            continue

        # 將關鍵點座標和分數打包成列表，方便後續處理
        # 這裡的 keypoints_data 會是 [(x, y, score), (x, y, score), ...]
        keypoints_data = []
        for i, (x, y) in enumerate(keypoints_coords):
            score = keypoint_scores[i] if i < len(keypoint_scores) else 0.0
            keypoints_data.append((int(x), int(y), score)) # 轉換為整數像素座標

        # 繪製關鍵點
        for kp_idx, (x, y, score) in enumerate(keypoints_data):
            if score > min_score_thresh:
                # 關鍵點顏色 (B, G, R) - 例如，綠色
                cv2.circle(frame, (x, y), point_radius, (0, 255, 0), -1)

        # 繪製骨架連接
        for connection in COCO_CONNECTIONS:
            start_kp_idx = connection[0]
            end_kp_idx = connection[1]

            if start_kp_idx < len(keypoints_data) and end_kp_idx < len(keypoints_data):
                x1, y1, s1 = keypoints_data[start_kp_idx]
                x2, y2, s2 = keypoints_data[end_kp_idx]

                if s1 > min_score_thresh and s2 > min_score_thresh:
                    # 連線顏色 (B, G, R) - 例如，黃色
                    cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 255), line_thickness)
    return frame

def render_video_with_pose(video_path: str, api_response_json: dict) -> tuple:
    """
    將姿態偵測結果渲染到原始影片的每一幀上，並返回幀列表和影片的 FPS。

    Args:
        video_path (str): 原始影片文件的路徑。
        api_response_json (dict): 從 API 獲得的完整 JSON 回應。

    Returns:
        tuple: (list, float) 包含每個渲染後幀的 NumPy 圖片陣列 (RGB 格式)，
               以及影片的幀率 (FPS)。如果影片無法讀取，則返回 ([], 0.0)。
    """
    rendered_frames = []
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return [], 0.0

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS) # 獲取影片的 FPS
    print(f"Video opened: {video_path}, Resolution: {frame_width}x{frame_height}, Frames: {total_frames}, FPS: {fps}")


    api_frames_data = {frame_data['frame_idx']: frame_data for frame_data in api_response_json.get('frames', [])}

    current_frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break # 影片讀取結束

        # 從 API 回應中獲取當前幀的姿態數據
        predictions_raw = api_frames_data.get(current_frame_idx, {}).get('predictions', [])

        # *** 修正這裡：根據您提供的數據結構，predictions 是一個包含一個列表的列表 ***
        # 例如: [[{'keypoints': ...}, {'keypoints': ...}]]
        # 我們需要取出內層的列表
        predictions_for_current_frame = []
        if predictions_raw and isinstance(predictions_raw, list):
            if predictions_raw and isinstance(predictions_raw[0], list):
                predictions_for_current_frame = predictions_raw[0]
            elif predictions_raw and isinstance(predictions_raw[0], dict):
                # 如果未來 API 結構改變，直接就是字典列表，這裡也可以兼容
                predictions_for_current_frame = predictions_raw

        # 在幀上繪製姿態
        rendered_frame = draw_pose_on_frame(
            frame.copy(), # 傳遞副本以避免修改原始幀
            predictions_for_current_frame
        )

        # 將 OpenCV 的 BGR 格式轉換為 Matplotlib 的 RGB 格式
        rendered_frames.append(cv2.cvtColor(rendered_frame, cv2.COLOR_BGR2RGB))
        current_frame_idx += 1

    cap.release()
    print(f"Finished rendering {len(rendered_frames)} frames.")
    return rendered_frames, fps # 同時返回幀列表和 FPS

def play_video_from_frames(frames: list, fps: float = 30.0, output_filename: str = 'output_pose_video.mp4'):
    """
    使用 Matplotlib 播放影片幀序列。在 Colab 環境中，將影片保存為 MP4 並嵌入顯示。

    Args:
        frames (list): 包含所有影片幀 (NumPy array) 的列表。
        fps (float): 影片的幀率，用於計算播放間隔。
        output_filename (str): 動畫將保存為指定文件名的 MP4 影片。
    """
    if not frames:
        print("No frames to play.")
        return

    interval_ms = 1000 / fps if fps > 0 else 40 # 計算每幀間隔，預設40ms

    fig, ax = plt.subplots(figsize=(frames[0].shape[1] / 100, frames[0].shape[0] / 100), dpi=100)
    ax.axis('off')
    img = ax.imshow(frames[0])

    def update(frame_idx):
        img.set_array(frames[frame_idx])
        return [img]

    ani = FuncAnimation(fig, update, frames=len(frames), interval=interval_ms, blit=True)

    print(f"Saving animation to {output_filename}...")
    try:
        # 嘗試保存動畫。在 Colab 中，ffmpeg 通常已預裝並在 PATH 中。
        # 'dpi' 參數可以調整輸出影片的解析度
        ani.save(output_filename, writer='ffmpeg', fps=fps, dpi=200)
        print(f"Animation saved as {output_filename}")

        # 在 Colab 中嵌入並顯示影片
        if os.path.exists(output_filename):
            display(Video(output_filename, embed=True, html_attributes="controls autoplay loop"))
        else:
            print(f"Warning: Output video file '{output_filename}' not found after saving attempt.")

    except Exception as e:
        print(f"Error saving and displaying animation: {e}")
        print("Please ensure FFmpeg is installed and accessible (e.g., 'sudo apt-get install ffmpeg' on Linux systems, or it's often pre-installed in Colab).")

    # 關閉繪圖，釋放記憶體
    plt.close(fig)

if __name__ == '__main__':
    # --- 測試區塊 ---
    # 這裡的 dummy_api_response 變數將直接使用您之前提供的 response.json() 內容
    # 因為我無法直接執行您的 API 調用，請確保您在運行此腳本前，
    # 已經將 response.json() 的實際結果賦值給 dummy_api_response。

    # 請務必將此替換為您實際的 response.json() 內容！
    dummy_api_response = response.json()

    # 替換為您的實際影片路徑
    video_file_path = '/content/pitch_0029.mp4' # 範例中的影片路徑

    # 運行渲染和播放
    print(f"Attempting to render video: {video_file_path}")
    rendered_frames, fps = render_video_with_pose(video_file_path, dummy_api_response)

    if rendered_frames:
        print("Playing rendered video...")
        # 調用修改後的函數，並指定輸出檔案名
        play_video_from_frames(rendered_frames, fps, output_filename='output_pose_video.mp4')
    else:
        print("No frames were rendered. Check video path and API response data.")